# Function calling

In [305]:
# preparando ambiente
import json
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Caminho lógico

Um problema dos modelos de linguagem é a atualização dos dados. No caso do GPT, os modelos não tem acesso a internet e sua informação sempre tem um teto temporal.
Uma maneira de lidar com essa questão é oferecer ao modelo algumas ferramentas (tools) para que possam buscar essas informações de outras maneiras. um exemplo, é oferecer ao modelo algumas funções que podem ser acessada com parâmetros.
Como estamos falando de um modelo de linguagem, essas funções funcionam melhor quando o nome descreve bem a sua funcionalidade, assim como os seus parâmetros.

Nesta seção vamos entender o código, passo-a-passo. Ao final você encontra um seção que **resume** a construção desse algoritmo.

### Função de apoio (tool)

Imagine que tenhamos uma base de dados com informações sobre a temperatura em determinados locais. Podemos utilizar essa informação no modelo, permitindo que ele acesse esses dados por meio de uma função. Vamos definir essa função como se segue. A função recebe dois parâmetros (local e a unidade) e retorna uma **string** de um dicionário com as chaves "local", "temperatura" e "unidade".

In [306]:
# definindo uma função que possa retornar a temperatura de 
# determinados locais

def obter_temperatura_atual(local, unidade="celsius"): # o nome da função é importante para que o modelo entenda o que ela faz
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )

Vamos testar algumas saídas:

In [307]:
test = obter_temperatura_atual('São Paulo')
print(test)

{"local": "S\u00e3o Paulo", "temperatura": "32", "unidade": "celsius"}


In [308]:
type(test)

str

In [309]:
test = obter_temperatura_atual('Penápolis')
print(test)

{"local": "Pen\u00e1polis", "temperatura": "unknown"}


Quando a função não conhece a temperatura do lugar ela devolve a string informando unknown.

### Testando o algorítmo com informações atuais
Agora, vamos fazer uma requisição da maneira que já conhecemos:

In [310]:
messagens = [{'role':'user','content':'Qual a temperatura em São Paulo neste momento?'}]

resposta = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messagens,
    max_tokens=100, 
    temperature=0,
)

Observando a resposta do modelo:

In [311]:
print(resposta.choices[0].message.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a temperatura atual em São Paulo. Recomendo verificar um site de meteorologia ou um aplicativo de clima para obter as informações mais recentes.


Percebemos que o modelo nos informa de que não tem informações em tempo real. 

### Entendo a lista de ferramentas (tools)
Seguindo o exemplo anterior, podemos permitir que o modelo utilize a nossa função `obter_temperatura_atual` indicando um parâmetro `tools` (uma lista de ferramentas disponíveis) e `tool_choice` (a maneira como o modelo pode escolher as ferramentas).

Antes, vamos então denifir a nossa lista de ferramentas, cada ferramenta disponível será informada no formato de dicionário:

In [312]:
# definindo a lista de tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]

In [313]:
# entendendo o objeto tools
print(type(tools), len(tools))

<class 'list'> 1


Percebemos que o tools é uma lista de 1 elemento (nossa função de obter a temperatura). Essa lista "explica" para o modelo o que é cada ferramenta disponibilizada, quais são seus parâmetros e outros detalhes.

Vamos explorar a função:

In [314]:
funcao = tools[0]
type(funcao)

dict

In [315]:
# keys da funcao
funcao.keys()

dict_keys(['type', 'function'])

As keys da função são `type` e `function`:

In [316]:
funcao['type']

'function'

In [317]:
funcao['function']

{'name': 'obter_temperatura_atual',
 'description': 'Obtém a temperatura atual em uma dada cidade',
 'parameters': {'type': 'object',
  'properties': {'local': {'type': 'string',
    'description': 'O nome da cidade. Ex: São Paulo'},
   'unidade': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
  'required': ['local']}}

In [318]:
funcao['function'].keys()

dict_keys(['name', 'description', 'parameters'])

In [319]:
# nome da função
funcao['function']['name']

'obter_temperatura_atual'

In [320]:
# descrição da função
funcao['function']['description']

'Obtém a temperatura atual em uma dada cidade'

In [321]:
# parâmetros da função
funcao['function']['parameters']

{'type': 'object',
 'properties': {'local': {'type': 'string',
   'description': 'O nome da cidade. Ex: São Paulo'},
  'unidade': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
 'required': ['local']}

In [322]:
# dentro dos parâmetros temos:
funcao['function']['parameters'].keys()

dict_keys(['type', 'properties', 'required'])

In [323]:
# tipo do parâmetro
funcao['function']['parameters']['type']

'object'

In [324]:
# propriedades do parâmetro
funcao['function']['parameters']['properties']

{'local': {'type': 'string', 'description': 'O nome da cidade. Ex: São Paulo'},
 'unidade': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}}

Dentro das propriedades dos parâmetros é que iremos indicar cada um dos parâmetros.

In [325]:
# parâmetros das funções
funcao['function']['parameters']['properties'].keys()

dict_keys(['local', 'unidade'])

E, dentro de cada um desses parâmetros, temos as informações:

In [326]:
# definições do local (nome da cidade)
print(funcao['function']['parameters']['properties']['local'].keys())
print(funcao['function']['parameters']['properties']['local']['type'])
print(funcao['function']['parameters']['properties']['local']['description'])

dict_keys(['type', 'description'])
string
O nome da cidade. Ex: São Paulo


In [327]:
# definições da unidade (de medida da temperatura)
print(funcao['function']['parameters']['properties']['unidade'].keys())
print(funcao['function']['parameters']['properties']['unidade']['type'])
print(funcao['function']['parameters']['properties']['unidade']['enum'])

dict_keys(['type', 'enum'])
string
['celsius', 'fahrenheit']


O `enum` é utilizada para definir categorias de uma variável.

### Utilizando as tools no modelo
Agora sim, estamos prontos para interagir com o modelo permitindo que ele acesse funções:

In [328]:
# vamos repetir a pergunta anterior
messages = [{'role':'user','content':'Qual a temperatura em São Paulo neste momento?'}]
resposta = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    max_tokens=100, 
    temperature=0,
)
print(resposta.choices[0].message.content)

Desculpe, mas não consigo fornecer informações em tempo real, como a temperatura atual em São Paulo. Recomendo verificar um site de meteorologia ou um aplicativo de clima para obter as informações mais recentes.


Agora, vamos refazer a pergunta passando os parâmetros com as `tools` e observar o que o modelo responde 

In [329]:
messages = [{'role':'user','content':'Qual a temperatura em São Paulo neste momento?'}]
resposta = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice='auto',
    messages=messages,
    max_tokens=100, 
    temperature=0,
)
print(resposta)

ChatCompletion(id='chatcmpl-A9KG1n4aKQNkv7z9oE7z5nQyfmyMx', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None))], created=1726786185, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_e9627b5346', usage=CompletionUsage(completion_tokens=20, prompt_tokens=83, total_tokens=103, completion_tokens_details={'reasoning_tokens': 0}))


Analisando a resposta do modelo, percebemos que não há mensagem de retorno e aparentemente o motivo foi relacionado a tools:

In [330]:
resposta.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None))

In [331]:
resposta.choices[0].finish_reason

'tool_calls'

#### Entendendo a resposta do modelo
Dentro do `message` agora há o `tool_calls`, que nada mais é do que o modelo solicitando que a função `obter_temperatura_atual`, seja chamada com o argumento `"local":"São Paulo"`

In [332]:
resposta.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]

#### Chamando as funções solicitadas
Então, precisamos chamar essa função com esse parâmetro e devolver esse valor para o modelo formular a sua resposta:

In [333]:
# armazenando as solicitações de chamada em um objeto
message_resp = resposta.choices[0].message
tool_calls = message_resp.tool_calls
print(message_resp)
print(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None)
[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]


In [334]:
# verificando se há chamadas solicitadas pelo modelo:
if tool_calls:
    print(tool_calls)

[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')]


In [335]:
# adicionando a mensagem de resposta na lista de mensagens
messages.append(message_resp)

In [336]:
messages

[{'role': 'user', 'content': 'Qual a temperatura em São Paulo neste momento?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None)]

In [337]:
len(messages)

2

In [338]:
messages[1]

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None)

In [339]:
# o objeto tool_calls é uma lista, porque o modelo pode pedir para chamar várias tools:
type(tool_calls)

list

In [340]:
# então, vamos iterar sobre cada uma das chamadas solicitadas:
for tool_call in tool_calls:
    print(tool_call)

ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')


Então, vamos iterar sobre cada uma das chamadas solicitadas:

In [341]:
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    print(function_name)

obter_temperatura_atual


Agora, precisamos executar essa função, pra isso também precisamos criar um dicionário que **leia** o nome da nossa função e **retorne** a função em si:

In [342]:
funcoes_disponiveis = {
        "obter_temperatura_atual": obter_temperatura_atual,
    }

In [343]:
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    function_to_call = funcoes_disponiveis[function_name]
    print(function_to_call)

<function obter_temperatura_atual at 0x000001D233E65BC0>


Também precisamos dos argumentos da função:

In [344]:
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    function_to_call = funcoes_disponiveis[function_name]
    function_args = json.loads(tool_call.function.arguments)
    print(function_args)

{'local': 'São Paulo'}


Agora, só nos falta chamar a função:

In [345]:
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    # pegar a função 
    function_to_call = funcoes_disponiveis[function_name]
    # argumentos da função
    function_args = json.loads(tool_call.function.arguments)
    # chamando a função:
    function_response = function_to_call(
                local=function_args.get("local"),
                unidade=function_args.get("unidade"),
            )
    print(function_response)

{"local": "S\u00e3o Paulo", "temperatura": "32", "unidade": null}


Com o resultado da função, vamos adicionar essa "mensagem" à nossa lista de mensagens:

In [346]:
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    # pegar a função 
    function_to_call = funcoes_disponiveis[function_name]
    # argumentos da função
    function_args = json.loads(tool_call.function.arguments)
    # chamando a função:
    function_response = function_to_call(
                local=function_args.get("local"),
                unidade=function_args.get("unidade"),
            )
    # adicionando a resposta da função à lista de mensagens
    messages.append(
            {
                "tool_call_id": tool_call.id, # importante passar o id da chamada
                "role": "tool", # informamos que é uma mensagem de tool (diferente do que fizemos até agora com user e assistant)
                "name": function_name,
                "content": function_response,
            }
        )
    print(messages)

[{'role': 'user', 'content': 'Qual a temperatura em São Paulo neste momento?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None), {'tool_call_id': 'call_7q4N0TLBep2iaZy1fdQRXdZW', 'role': 'tool', 'name': 'obter_temperatura_atual', 'content': '{"local": "S\\u00e3o Paulo", "temperatura": "32", "unidade": null}'}]


#### Usando o resultado das chamadas na requisição

Após finalizar de adicionar todas as respostas das chamadas, podemos realizar uma nova "pergunta" ao GPT:

In [347]:
segunda_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
    )

E, finalmente, podemos ver a resposta do modelo:

In [348]:
segunda_resposta.choices[0].message

ChatCompletionMessage(content='A temperatura em São Paulo neste momento é de 32 graus Celsius.', role='assistant', function_call=None, tool_calls=None, refusal=None)

In [349]:
segunda_resposta.choices[0].message.content

'A temperatura em São Paulo neste momento é de 32 graus Celsius.'

No fim das contas, o modelo foi capaz de incorporar novas informações (por meio de uma função) ao responder o usuário.
Neste exemplo utilizei o parâmetro `temperature = 0`, mas funciona com outros valores também:

In [350]:
segunda_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=1,
    )
segunda_resposta.choices[0].message.content

'A temperatura em São Paulo neste momento é de 32°C.'

#### Mais de uma chamada para funções
E o que acontece de pedir a temperatura de duas cidades?

In [351]:
# adicionando uma nova request com duas outras cidades
messages.append({'role':'user','content':'E qual a temperatura em Porto Alegre e Rio de Janeiro?'})

In [352]:
messages

[{'role': 'user', 'content': 'Qual a temperatura em São Paulo neste momento?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None),
 {'tool_call_id': 'call_7q4N0TLBep2iaZy1fdQRXdZW',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "S\\u00e3o Paulo", "temperatura": "32", "unidade": null}'},
 {'role': 'user',
  'content': 'E qual a temperatura em Porto Alegre e Rio de Janeiro?'}]

In [353]:
# fazendo a solicitação: 
terceira_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
        tools=tools,
        tool_choice='auto'
    )
terceira_resposta

ChatCompletion(id='chatcmpl-A9KG3AM9aAoqbHi6gJJcHgmgjONgF', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LX8yUw0HvS8iVr6QblTUhZgJ', function=Function(arguments='{"local": "Porto Alegre"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_XwH2auyun5HPi2HTcgSzBZd1', function=Function(arguments='{"local": "Rio de Janeiro"}', name='obter_temperatura_atual'), type='function')], refusal=None))], created=1726786187, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_1bb46167f9', usage=CompletionUsage(completion_tokens=58, prompt_tokens=157, total_tokens=215, completion_tokens_details={'reasoning_tokens': 0}))

Novamente, o modelo nos solicita para rodar as funções, dessa vez, a lista de funções a serem rodadas tem dois objetos. A primeira com o argumento `"local": "Porto Alegre"` e a segunda com `"local": "Rio de Janeiro"`.

In [354]:
message_resp = terceira_resposta.choices[0].message
tool_calls = message_resp.tool_calls
print(message_resp)
len(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LX8yUw0HvS8iVr6QblTUhZgJ', function=Function(arguments='{"local": "Porto Alegre"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_XwH2auyun5HPi2HTcgSzBZd1', function=Function(arguments='{"local": "Rio de Janeiro"}', name='obter_temperatura_atual'), type='function')], refusal=None)


2

In [355]:
tool_calls[0]

ChatCompletionMessageToolCall(id='call_LX8yUw0HvS8iVr6QblTUhZgJ', function=Function(arguments='{"local": "Porto Alegre"}', name='obter_temperatura_atual'), type='function')

In [356]:
tool_calls[1]

ChatCompletionMessageToolCall(id='call_XwH2auyun5HPi2HTcgSzBZd1', function=Function(arguments='{"local": "Rio de Janeiro"}', name='obter_temperatura_atual'), type='function')

Vamos chamar novamente essas funções com os novos parâmetros:

In [357]:
# adicionando a mensagem de chamada das requisições
messages.append(message_resp)
messages

[{'role': 'user', 'content': 'Qual a temperatura em São Paulo neste momento?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_7q4N0TLBep2iaZy1fdQRXdZW', function=Function(arguments='{"local":"São Paulo"}', name='obter_temperatura_atual'), type='function')], refusal=None),
 {'tool_call_id': 'call_7q4N0TLBep2iaZy1fdQRXdZW',
  'role': 'tool',
  'name': 'obter_temperatura_atual',
  'content': '{"local": "S\\u00e3o Paulo", "temperatura": "32", "unidade": null}'},
 {'role': 'user',
  'content': 'E qual a temperatura em Porto Alegre e Rio de Janeiro?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_LX8yUw0HvS8iVr6QblTUhZgJ', function=Function(arguments='{"local": "Porto Alegre"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_XwH2auyun5HPi2HTcgSzBZd1', function=Function(arguments='{"local": "

In [358]:
# chamando as funções
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    # pegar a função 
    function_to_call = funcoes_disponiveis[function_name]
    # argumentos da função
    function_args = json.loads(tool_call.function.arguments)
    # chamando a função:
    function_response = function_to_call(
                local=function_args.get("local"),
                unidade=function_args.get("unidade"),
            )
    # adicionando a resposta da função à lista de mensagens
    messages.append(
            {
                "tool_call_id": tool_call.id, # importante passar o id da chamada
                "role": "tool", # informamos que é uma mensagem de tool (diferente do que fizemos até agora com user e assistant)
                "name": function_name,
                "content": function_response,
            }
        )
# nova requisição
quarta_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
    )


In [359]:
print(quarta_resposta.choices[0].message.content)

Atualmente, as temperaturas são as seguintes:

- São Paulo: 32°C
- Porto Alegre: 25°C
- Rio de Janeiro: 35°C


#### Parâmetros fora do escopo das funções
E o que acontece se adicionarmos uma cidade fora do escopo da função?

In [360]:
# adicionando uma nova request com cidade desconhecida na função
messages.append({'role':'user','content':'E qual a temperatura em Penápolis?'})

# fazendo a solicitação: 
resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
        tools=tools,
        tool_choice='auto'
    )

message_resp = resposta.choices[0].message
tool_calls = message_resp.tool_calls

# adicionando a mensagem de chamada das requisições
messages.append(message_resp)

# chamando as funções
for tool_call in tool_calls:
    # pegar o nome da função
    function_name = tool_call.function.name
    # pegar a função 
    function_to_call = funcoes_disponiveis[function_name]
    # argumentos da função
    function_args = json.loads(tool_call.function.arguments)
    # chamando a função:
    function_response = function_to_call(
                local=function_args.get("local"),
                unidade=function_args.get("unidade"),
            )
    # adicionando a resposta da função à lista de mensagens
    messages.append(
            {
                "tool_call_id": tool_call.id, # importante passar o id da chamada
                "role": "tool", # informamos que é uma mensagem de tool (diferente do que fizemos até agora com user e assistant)
                "name": function_name,
                "content": function_response,
            }
        )
# nova requisição
quarta_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
    )

In [361]:
print(quarta_resposta.choices[0].message.content)

Atualmente, não consegui obter a temperatura em Penápolis. Se precisar de informações sobre outra localidade ou mais detalhes, estou à disposição!


## Resumindo
1. Criar as funções de apoio (as funções devem retornar **strings** para que o modelo seja capaz de entender)
1. Adicionar as funções à uma lista `tools`
1. Criar um dicionário com `"function_name":function`
1. Criar o objeto `messages` com a mensagem inicial
1. Fazer a requisição com o parâmetro `tools`
1. Se houver solicitação do modelo para rodar funções:
    1. Adicionar a resposta ao `messages`
    1. Rodar as funções para cada chamada `tool_calls` solicitada pelo modelo e adicionar as respostas ao `messages`
1. Refazer a requisição
1. Ler a resposta

In [362]:
# 1. Criar as funções de apoio 
def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )

# 2. Adicionar as funções à uma lista `tools`
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]

# 3. Criar um dicionário com `function_name:function`
funcoes_disponiveis = {
        "obter_temperatura_atual": obter_temperatura_atual,
    }

In [363]:
# 4. Criar o objeto `messages` com a mensagem inicial
messages = [{'role':'user','content':'Qual a temperatura em São Paulo, no Rio de Janeiro e em Penápolis neste momento?'}]

# 5. Fazer a requisição com o parâmetro `tools`
resposta = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice='auto',
    messages=messages,
    max_tokens=100, 
    temperature=0,
)

message_resp = resposta.choices[0].message
tool_calls = message_resp.tool_calls

# 6. Se houver solicitação do modelo para rodar funções:
if tool_calls:
    # 6.1 Adicionar a resposta ao `messages`
    messages.append(message_resp)
    # 6.2 Rodar as funções para cada chamada `tool_calls` solicitada pelo modelo e ...
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            local=function_args.get("local"),
            unidade=function_args.get("unidade"),
        )
        # ... adicionar as respostas ao `messages`
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

# 7. Refazer a requisição
nova_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
    )

# 8. Ler a resposta
print(nova_resposta.choices[0].message.content)

Atualmente, a temperatura é a seguinte:

- São Paulo: 32°C
- Rio de Janeiro: 35°C
- Penápolis: Informação de temperatura não disponível. 

Se precisar de mais alguma coisa, é só avisar!
